In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd /home/aditya/git/RCNN_Pneumonia
%env PROJECT_PATH /home/aditya/git/RCNN_Pneumonia

/home/aditya/git/RCNN_Pneumonia
env: PROJECT_PATH=/home/aditya/git/RCNN_Pneumonia


In [3]:
from utils.envs import *
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from model.dataset import PneumoniaDataset
from mrcnn.model import log
from mrcnn import utils
from model.config import InferenceConfig
from mrcnn import visualize
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [4]:
from mrcnn import model as modellib
from utils.vis import get_ax

In [5]:
train_dataset = PneumoniaDataset(data_dir, 'train')
dev_dataset = PneumoniaDataset(data_dir, 'dev')
val_dataset = PneumoniaDataset(data_dir, 'val')
test_dataset = PneumoniaDataset(data_dir, 'test')

In [6]:
train_dataset.prepare()
dev_dataset.prepare()
val_dataset.prepare()
test_dataset.prepare()

In [7]:
config = InferenceConfig()

In [8]:
# Create model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          model_dir=logs_dir,
                          config=config)

In [9]:
n_epoch = '21'

In [10]:
weights_path = '/home/aditya/git/RCNN_Pneumonia/logs/pneumonia20181026T1746/mask_rcnn_pneumonia_00{}.h5'.format(n_epoch)
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /home/aditya/git/RCNN_Pneumonia/logs/pneumonia20181026T1746/mask_rcnn_pneumonia_0021.h5
Re-starting from epoch 21


In [11]:
result_95 = []
result_97 = []
result_99 = []
for idx in tqdm(test_dataset.image_ids):
    img = test_dataset.load_image(idx)
    patient_id = test_dataset.image_info[idx]['patient_id']
    prediction = model.detect([img])
    rois = prediction[0]['rois']
    score = prediction[0]['scores']
    
    predictionString = ''
    for i in range(len(score)):
        if score[i] > 0.95:
            x1, y1, x2, y2 = int(rois[i][1]), int(rois[i][0]), int(rois[i][3]), int(rois[i][2])
            x, y, width, height = x1, y1, x2-x1, y2-y1
            predictionString = predictionString + '{} {} {} {} {} '.format(score[i], x, y, width, height)
    result_95.append({
        'patientId' : patient_id,
        'predictionString' : predictionString.strip()
    })
    
    predictionString = ''
    for i in range(len(score)):
        if score[i] > 0.97:
            x1, y1, x2, y2 = int(rois[i][1]), int(rois[i][0]), int(rois[i][3]), int(rois[i][2])
            x, y, width, height = x1, y1, x2-x1, y2-y1
            predictionString = predictionString + '{} {} {} {} {} '.format(score[i], x, y, width, height)
    result_97.append({
        'patientId' : patient_id,
        'predictionString' : predictionString.strip()
    })
    
    predictionString = ''
    for i in range(len(score)):
        if score[i] > 0.99:
            x1, y1, x2, y2 = int(rois[i][1]), int(rois[i][0]), int(rois[i][3]), int(rois[i][2])
            x, y, width, height = x1, y1, x2-x1, y2-y1
            predictionString = predictionString + '{} {} {} {} {} '.format(score[i], x, y, width, height)
    result_99.append({
        'patientId' : patient_id,
        'predictionString' : predictionString.strip()
    })

In [12]:
iteration = 7

In [13]:
result_95_df = pd.DataFrame(result_95)
result_97_df = pd.DataFrame(result_97)
result_99_df = pd.DataFrame(result_99)

result_95_path = os.path.join(output_path, 'result_{}_{}_{}.csv').format(iteration, n_epoch, 0.95)
result_97_path = os.path.join(output_path, 'result_{}_{}_{}.csv').format(iteration, n_epoch, 0.97)
result_99_path = os.path.join(output_path, 'result_{}_{}_{}.csv').format(iteration, n_epoch, 0.99)

result_95_df.to_csv(result_95_path, index=False)
result_97_df.to_csv(result_97_path, index=False)
result_99_df.to_csv(result_99_path, index=False)